In [ ]:
import json
from pathlib import Path
import numpy as np
from collections import defaultdict
from sort.sort import Sort  # pip install sort-tracker or clone from abewley/sort (cloning preferred)


In [ ]:
def coco_bbox_to_sort_bbox(center_x, center_y, width, height):
    x1 = center_x - width / 2
    y1 = center_y - height / 2
    x2 = center_x + width / 2
    y2 = y1 + height
    return [x1, y1, x2, y2]


In [ ]:
DATASET_DIR = Path("dataset_jpg/dataset")
ANN_FILE = DATASET_DIR / "annotations.json"

MAX_AGE = 30  # 1 to 30, 1 standard
MIN_HITS = 3  # 1 to 5, 3 standard
IOU_THRESHOLD = 0.3  # 0.0 to 1.0, 3.0 standard

with ANN_FILE.open("r") as f:
    coco_data = json.load(f)

annotations = coco_data['annotations']
images = coco_data['images']

# Build image_id → image_info
image_id_to_info = {img['id']: img for img in images}

# Group annotations by video_id and then by frame_id
video_frames = defaultdict(lambda: defaultdict(list))
for ann in annotations:
    img_info = image_id_to_info[ann['image_id']]
    video_id = ann['video_id']
    frame_id = img_info['frame_id']
    bbox = ann['bbox']
    category_id = ann['category_id']

    # Convert [center_x, center_y, w, h] → [x1, y1, x2, y2]
    x1, y1, x2, y2 = coco_bbox_to_sort_bbox(*bbox)
    video_frames[video_id][frame_id].append([x1, y1, x2, y2, 1.0])  # Confidence = 1.0


In [ ]:
tracking_results = []  # To hold all frame tracking results

for video_id, frames in video_frames.items():
    tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
    sorted_frames = sorted(frames.items())

    for frame_id, detections in sorted_frames:
        dets_np = np.array(detections)
        tracks = tracker.update(dets_np)

        for track in tracks:
            x1, y1, x2, y2, track_id = track
            tracking_results.append({
                'video_id': video_id,
                'frame_id': frame_id,
                'track_id': int(track_id),
                'bbox': [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
            })


In [ ]:
import csv

with open('sort_tracking_results.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['video_id', 'frame_id', 'track_id', 'bbox'])
    writer.writeheader()
    for row in tracking_results:
        writer.writerow(row)


# Evaluation

In [ ]:
def compute_iou_matrix(gt_boxes, pred_boxes):
    """
    Compute an IoU matrix between two sets of boxes.
    Each box is [x, y, w, h]
    Returns a cost matrix where cost = 1 - IoU (for motmetrics)
    """
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)), dtype=np.float32)

    for i, (x1, y1, w1, h1) in enumerate(gt_boxes):
        xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
        area_gt = w1 * h1

        for j, (x2, y2, w2, h2) in enumerate(pred_boxes):
            xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2
            area_pred = w2 * h2

            # Intersection
            inter_x1 = max(xa1, xb1)
            inter_y1 = max(ya1, yb1)
            inter_x2 = min(xa2, xb2)
            inter_y2 = min(ya2, yb2)

            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            union_area = area_gt + area_pred - inter_area
            iou = inter_area / union_area if union_area > 0 else 0.0
            iou_matrix[i, j] = 1.0 - iou  # for motmetrics: cost = 1 - IoU

    return iou_matrix

In [ ]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict
from sort.sort import Sort
import motmetrics as mm

# Load your COCO-style explanation.json
# with open("explanation.json", "r") as f:
#     data = json.load(f)

# annotations = coco_data["annotations"]
# images = coco_data["images"]
videos = coco_data["videos"]

# Build index
image_by_id = {img["id"]: img for img in images}
anns_by_video = defaultdict(list)
for ann in annotations:
    anns_by_video[ann["video_id"]].append(ann)

# Init MOT accumulator
mm.lap.default_solver = 'lap'
acc = mm.MOTAccumulator(auto_id=True)

# Process a single video (repeat in loop for all) # ToDo: Make loop over all videos
video_id = 1  # iterate from 1 to 274
video_anns = anns_by_video[video_id]
tracker = Sort()

# Group annotations by frame
anns_by_frame = defaultdict(list)
for ann in video_anns:
    img = image_by_id[ann["image_id"]]
    anns_by_frame[img["frame_id"]].append(ann)

gt_all, pred_all = [], []

# Sort by frame_id
for frame_id in sorted(anns_by_frame.keys()):
    anns = anns_by_frame[frame_id]

    # Build detection list for SORT: [x1, y1, x2, y2, conf]
    dets = []
    gt_ids = []
    gt_boxes = []

    for ann in anns:
        x_center, y_center, w, h = ann["bbox"]
        x1, y1 = x_center - w / 2, y_center - h / 2
        x2, y2 = x1 + w, y1 + h
        dets.append([x1, y1, x2, y2, 1.0])  # dummy conf
        gt_ids.append(ann["cell_id"])
        gt_boxes.append([x1, y1, w, h])

        # Save GT
        gt_all.append([frame_id + 1, ann["cell_id"], x1, y1, w, h])  # 1-based frame

    dets_np = np.array(dets)
    tracks = tracker.update(dets_np)

    # Save predictions for motmetrics
    # pred_boxes = [trk["bbox"] for trk in tracking_results[(video_id, frame_id)]]
    # pred_ids = [trk["track_id"] for trk in tracking_results[(video_id, frame_id)]]
    pred_ids = []
    pred_boxes = []
    for track in tracks:
        x1, y1, x2, y2, track_id = track
        w, h = x2 - x1, y2 - y1
        pred_all.append([frame_id + 1, int(track_id), x1, y1, w, h])
        pred_ids.append(track_id)
        pred_boxes.append([x1, y1, w, h])

    # Match predictions to GT using bbox IoU
    from motmetrics.utils import iou_matrix

    if gt_boxes and pred_boxes:
        distances = compute_iou_matrix(gt_boxes, pred_boxes)
        distances[distances > (1 - 0.5)] = np.nan  # Apply IoU threshold (max_iou = 0.5)
        acc.update(gt_ids, pred_ids, distances)
    else:
        acc.update(gt_ids, [], np.array([]))  # no pred

# Save CSVs (optional)
pd.DataFrame(gt_all, columns=["frame", "id", "x", "y", "w", "h"]).to_csv("gt.csv", index=False)
pd.DataFrame(pred_all, columns=["frame", "id", "x", "y", "w", "h"]).to_csv("pred.csv", index=False)

# Evaluate
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name='SORT')
print(mm.io.render_summary(summary, formatters=mh.formatters, namemap=mm.io.motchallenge_metric_names))
# summary = mh.compute(acc, name='SORT')
# print(mm.io.render_summary(summary, formatters=mh.formatters))
print(f"Processed {len(gt_all)} frames")


In [ ]:
%matplotlib inline
import os
import json
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

# Load annotations
# with open("explanation.json") as f:
#     data = json.load(f)

# annotations = data["annotations"]
# images = data["images"]
image_by_id = {img["id"]: img for img in images}

# Map annotations per image
from collections import defaultdict
anns_by_image = defaultdict(list)
for ann in annotations:
    anns_by_image[ann["image_id"]].append(ann)

# Load GT and SORT predictions
import pandas as pd
gt_df = pd.read_csv("gt.csv")    # columns: frame, id, x, y, w, h
pred_df = pd.read_csv("pred.csv")  # same format with SORT track_ids

def draw_frame(video_id, frame_id, img_dir="dataset_jpg/dataset"):
    # Find image metadata
    image_info = next(img for img in images if img["video_id"] == video_id and img["frame_id"] == frame_id)
    image_id = image_info["id"]
    filename = os.path.join(img_dir, f"{video_id:03d}/images", os.path.basename(image_info["file_name"]))

    # Load image
    img = Image.open(filename).convert("RGB")
    draw = ImageDraw.Draw(img)

    # Load ground truth boxes
    for ann in anns_by_image[image_id]:
        x_center, y_center, w, h = ann["bbox"]
        x1 = x_center - w / 2
        y1 = y_center - h / 2
        x2 = x1 + w
        y2 = y1 + h
        category = ann["category_id"]  # 1 = living, 2 = dead
        cell_id = ann["cell_id"]

        color = "green" if category == 1 else "red"
        draw.rectangle([x1, y1, x2, y2], outline=color, width=2)
        draw.text((x1, y1 - 10), f"GT:{cell_id}", fill=color)

    # Load SORT predictions
    preds = pred_df[pred_df["frame"] == frame_id + 1]  # 1-based in pred.csv
    for _, row in preds.iterrows():
        x, y, w, h = row[2:6]
        x1, y1, x2, y2 = x, y, x + w, y + h
        track_id = int(row["id"])
        draw.rectangle([x1, y1, x2, y2], outline="blue", width=2)
        draw.text((x1, y2 + 2), f"SORT:{track_id}", fill="blue")

    # Show image
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.title(f"Video {video_id:03d}, Frame {frame_id}")
    plt.axis("off")
    plt.show()

# Example: Show frame 5 of video 1
draw_frame(video_id=1, frame_id=100)

# NEW VERSION

In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
from sort.sort import Sort
import motmetrics as mm

DATASET_DIR = Path("dataset_jpg/dataset")
ANN_FILE = DATASET_DIR / "annotations.json"

# === CONFIG ===
MAX_AGE = 1
MIN_HITS = 3
IOU_THRESHOLD = 0.3

# === Load Annotations ===
with ANN_FILE.open("r") as f:
    data = json.load(f)

annotations = data["annotations"]
images = data["images"]
image_by_id = {img["id"]: img for img in images}

# Group annotations by (video, frame)
video_frames = defaultdict(lambda: defaultdict(list))
for ann in annotations:
    img = image_by_id[ann["image_id"]]
    video_id = ann["video_id"]
    frame_id = img["frame_id"]

    x, y, w, h = ann["bbox"]
    x1, y1 = x - w / 2, y - h / 2
    x2, y2 = x1 + w, y1 + h
    bbox = [x1, y1, x2, y2, 1.0]  # dummy confidence

    video_frames[video_id][frame_id].append(bbox)

# === Tracking with SORT ===
tracking_results = []

for video_id, frames in video_frames.items():
    tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
    sorted_frames = sorted(frames.items())

    for frame_id, detections in sorted_frames:
        dets_np = np.array(detections)
        tracks = tracker.update(dets_np)

        for track in tracks:
            x1, y1, x2, y2, track_id = track
            tracking_results.append({
                "video_id": video_id,
                "frame_id": frame_id,
                "track_id": int(track_id),
                "bbox": [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
            })

# === Organize GT and Predictions for Evaluation ===

# Build ground truth per video/frame
gt_by_video_frame = defaultdict(list)
for ann in annotations:
    img = image_by_id[ann["image_id"]]
    x, y, w, h = ann["bbox"]
    x1, y1 = x - w / 2, y - h / 2
    video_id = ann["video_id"]
    frame_id = img["frame_id"]
    gt_by_video_frame[(video_id, frame_id)].append((ann["cell_id"], [x1, y1, w, h]))

# Group predictions
pred_by_video_frame = defaultdict(list)
for trk in tracking_results:
    pred_by_video_frame[(trk["video_id"], trk["frame_id"])].append((trk["track_id"], trk["bbox"]))

# === Custom IoU matrix ===
def compute_iou_matrix(gt_boxes, pred_boxes):
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)), dtype=np.float32)
    for i, (x1, y1, w1, h1) in enumerate(gt_boxes):
        xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
        area_gt = w1 * h1

        for j, (x2, y2, w2, h2) in enumerate(pred_boxes):
            xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2
            area_pred = w2 * h2

            inter_x1 = max(xa1, xb1)
            inter_y1 = max(ya1, yb1)
            inter_x2 = min(xa2, xb2)
            inter_y2 = min(ya2, yb2)

            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            union = area_gt + area_pred - inter_area
            iou = inter_area / union if union > 0 else 0.0
            iou_matrix[i, j] = 1.0 - iou  # motmetrics expects cost
    return iou_matrix

# === MOT Evaluation ===
acc = mm.MOTAccumulator(auto_id=True)

for key in sorted(gt_by_video_frame.keys()):
    gt = gt_by_video_frame[key]
    pred = pred_by_video_frame.get(key, [])

    gt_ids = [g[0] for g in gt]
    gt_boxes = [g[1] for g in gt]

    pred_ids = [p[0] for p in pred]
    pred_boxes = [p[1] for p in pred]

    if gt_boxes and pred_boxes:
        distances = compute_iou_matrix(gt_boxes, pred_boxes)
        distances[distances > (1 - 0.5)] = np.nan  # IoU threshold = 0.5
        acc.update(gt_ids, pred_ids, distances)
    else:
        acc.update(gt_ids, pred_ids, np.empty((len(gt_ids), len(pred_ids))))

# === Print Metrics ===
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name="SORT")
print(mm.io.render_summary(summary, formatters=mh.formatters, namemap=mm.io.motchallenge_metric_names))

print(f"\n✅ Processed {acc.events['FrameId'].nunique()} frames")


In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
from sort.sort import Sort
import motmetrics as mm
import gc

# === SET PARAMETER ===
DATASET_DIR = Path("D:/Maxi/Cell_Tracking_Project_6/dataset_jpg/dataset") # path to the dataset
ANN_FILE = DATASET_DIR / "annotations.json"

# === MODEL CONFIG ===
MAX_AGE = 1
MIN_HITS = 3
IOU_THRESHOLD = 0.3

# === Folders to Process ===
START_FOLDER = 1
END_FOLDER = 10


# === Custom IoU matrix ===
def compute_iou_matrix(gt_boxes, pred_boxes):
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)), dtype=np.float32)
    for i, (x1, y1, w1, h1) in enumerate(gt_boxes):
        # Get the coordinates for the ground truth rectangle
        xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
        area_gt = w1 * h1

        for j, (x2, y2, w2, h2) in enumerate(pred_boxes):
            # Get the coordinates for the prediction rectangle
            xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2
            area_pred = w2 * h2

            # compute coordinates for the area that intersects between ground truth and prediction
            inter_x1 = max(xa1, xb1)
            inter_y1 = max(ya1, yb1)
            inter_x2 = min(xa2, xb2)
            inter_y2 = min(ya2, yb2)

            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            # calculate IoU and output for motmetrics
            union = area_gt + area_pred - inter_area
            iou = inter_area / union if union > 0 else 0.0
            iou_matrix[i, j] = 1.0 - iou  # motmetrics expects cost
    return iou_matrix

# Initialize MOTAccumulator once
acc = mm.MOTAccumulator(auto_id=True) # Keep auto_id=True

# === Load Annotations - Optimized Reading (Initial Pass for Mapping) ===
print("Loading initial annotations for mapping...")
with ANN_FILE.open("r") as f:
    data = json.load(f)

annotations_full = data["annotations"]
images = data["images"]
image_info_by_id = {img["id"]: (img["video_id"], img["frame_id"]) for img in images}
all_video_ids_from_annotations = sorted(list(set(img["video_id"] for img in images)))

# Garbage collector
del data
gc.collect()
print("Loading completed.")

In [ ]:
# === Filter video_ids based on folder numbers ===
video_ids_to_process = []
for vid_id in all_video_ids_from_annotations:
    try:
        vid_num = int(vid_id)
        if START_FOLDER <= vid_num <= END_FOLDER:
             video_ids_to_process.append(vid_id)
    except ValueError:
        print(f"Skipping video_id '{vid_id}' as it's not a numeric ID for folder filtering.")
        continue

print(f"Total videos to process based on folder selection: {len(video_ids_to_process)}")


for video_idx, video_id in enumerate(video_ids_to_process):
    print(f"Processing video {video_idx + 1}/{len(video_ids_to_process)}: Video ID {video_id}", end='\r')

    video_annotations_by_frame = defaultdict(list)
    video_tracking_results = []

    for ann in annotations_full:
        img_id = ann["image_id"]
        if img_id in image_info_by_id:
            vid_id_from_img, frame_id = image_info_by_id[img_id]
            if vid_id_from_img == video_id:
                x, y, w, h = ann["bbox"]
                x1, y1 = x - w / 2, y - h / 2
                x2, y2 = x1 + w, y1 + h
                bbox_for_sort = [x1, y1, x2, y2, 1.0]
                video_annotations_by_frame[frame_id].append({
                    "cell_id": ann["cell_id"],
                    "bbox_gt": [x1, y1, w, h],
                    "bbox_sort": bbox_for_sort
                })

    tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
    sorted_frames = sorted(video_annotations_by_frame.items())

    for frame_id, frame_data_list in sorted_frames:
        detections_for_sort = np.array([item["bbox_sort"] for item in frame_data_list])
        if detections_for_sort.size > 0:
            tracks = tracker.update(detections_for_sort)

            for track in tracks:
                x1, y1, x2, y2, track_id = track
                video_tracking_results.append({
                    "frame_id": frame_id,
                    "track_id": int(track_id),
                    "bbox": [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
                })

    gt_by_frame = defaultdict(list)
    for frame_id, frame_data_list in video_annotations_by_frame.items():
        for ann_item in frame_data_list:
            gt_by_frame[frame_id].append((ann_item["cell_id"], ann_item["bbox_gt"]))

    pred_by_frame = defaultdict(list)
    for trk in video_tracking_results:
        pred_by_frame[trk["frame_id"]].append((trk["track_id"], trk["bbox"]))

    all_frames_in_video = sorted(list(set(list(gt_by_frame.keys()) + list(pred_by_frame.keys()))))

    for frame_id in all_frames_in_video:
        gt = gt_by_frame.get(frame_id, [])
        pred = pred_by_frame.get(frame_id, [])

        gt_ids = [g[0] for g in gt]
        gt_boxes = [g[1] for g in gt]

        pred_ids = [p[0] for p in pred]
        pred_boxes = [p[1] for p in pred]

        if gt_boxes or pred_boxes:
            if gt_boxes and pred_boxes:
                distances = compute_iou_matrix(gt_boxes, pred_boxes)
                distances[distances > (1 - 0.5)] = np.nan
                acc.update(gt_ids, pred_ids, distances)
            else:
                acc.update(gt_ids, pred_ids, np.empty((len(gt_ids), len(pred_ids))))

    # Garbage collector
    del video_annotations_by_frame
    del video_tracking_results
    del gt_by_frame
    del pred_by_frame
    gc.collect()

# === Print Metrics ===
print("\n--- Final Metrics --------------------------------------------------------------------")
mh = mm.metrics.create()
summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name="SORT")
print(mm.io.render_summary(summary, formatters=mh.formatters, namemap=mm.io.motchallenge_metric_names))
print("----------------------------------------------------------------------------------------")

In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
from sort.sort import Sort
import motmetrics as mm
import gc

DATASET_DIR = Path("D:/Maxi/Cell_Tracking_Project_6/dataset_jpg/dataset")
ANN_FILE = DATASET_DIR / "annotations.json"

# === Custom IoU matrix (remains the same) ===
def compute_iou_matrix(gt_boxes, pred_boxes):
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)), dtype=np.float32)
    for i, (x1, y1, w1, h1) in enumerate(gt_boxes):
        xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
        area_gt = w1 * h1

        for j, (x2, y2, w2, h2) in enumerate(pred_boxes):
            xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2
            area_pred = w2 * h2

            inter_x1 = max(xa1, xb1)
            inter_y1 = max(ya1, yb1)
            inter_x2 = min(xa2, xb2)
            inter_y2 = min(ya2, yb2)

            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            union = area_gt + area_pred - inter_area
            iou = inter_area / union if union > 0 else 0.0
            iou_matrix[i, j] = 1.0 - iou
    return iou_matrix

# === Main processing function ===
def run_tracking_and_evaluation(params, annotations_full, image_info_by_id, all_video_ids_from_annotations):
    """
    Runs the SORT tracking and motmetrics evaluation with given parameters.

    Args:
        params (dict): A dictionary containing 'name', 'MAX_AGE', 'MIN_HITS', 'IOU_THRESHOLD',
                       'START_FOLDER', 'END_FOLDER'.
        annotations_full (list): All annotations loaded from the JSON file.
        image_info_by_id (dict): Mapping from image_id to (video_id, frame_id).
        all_video_ids_from_annotations (list): List of all unique video IDs from annotations.

    Returns:
        tuple: (summary_df, name_of_run, total_processed_frames)
               summary_df: motmetrics summary DataFrame
               name_of_run: Name of the current parameter set
               total_processed_frames: Number of unique frames processed
    """

    name_of_run = params['name']
    MAX_AGE = params['MAX_AGE']
    MIN_HITS = params['MIN_HITS']
    IOU_THRESHOLD = params['IOU_THRESHOLD']
    START_FOLDER = params['START_FOLDER']
    END_FOLDER = params['END_FOLDER']

    print(f"\n--- Starting run: {name_of_run} ---")
    print(f"Parameters: MAX_AGE={MAX_AGE}, MIN_HITS={MIN_HITS}, IOU_THRESHOLD={IOU_THRESHOLD}")
    print(f"Folders: {START_FOLDER} to {END_FOLDER}")

    acc = mm.MOTAccumulator(auto_id=True)

    video_ids_to_process = []
    for vid_id in all_video_ids_from_annotations:
        try:
            vid_num = int(vid_id)
            if START_FOLDER <= vid_num <= END_FOLDER:
                 video_ids_to_process.append(vid_id)
        except ValueError:
            # print(f"Skipping video_id '{vid_id}' as it's not a numeric ID for folder filtering.")
            continue # Suppress this print for multiple runs to avoid clutter

    print(f"Total videos to process for this run: {len(video_ids_to_process)}")

    total_processed_frames = 0 # initialize counter to keep track of total frames

    for video_idx, video_id in enumerate(video_ids_to_process):
        # print(f"Processing video {video_idx + 1}/{len(video_ids_to_process)}: Video ID {video_id}") # Too verbose for multiple runs

        video_annotations_by_frame = defaultdict(list)
        video_tracking_results = []

        for ann in annotations_full:
            img_id = ann["image_id"]
            if img_id in image_info_by_id:
                vid_id_from_img, frame_id = image_info_by_id[img_id]
                if vid_id_from_img == video_id:
                    x, y, w, h = ann["bbox"]
                    x1, y1 = x - w / 2, y - h / 2
                    x2, y2 = x1 + w, y1 + h
                    bbox_for_sort = [x1, y1, x2, y2, 1.0]
                    video_annotations_by_frame[frame_id].append({
                        "cell_id": ann["cell_id"],
                        "bbox_gt": [x1, y1, w, h],
                        "bbox_sort": bbox_for_sort
                    })

        tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
        sorted_frames = sorted(video_annotations_by_frame.items())

        for frame_id, frame_data_list in sorted_frames:
            detections_for_sort = np.array([item["bbox_sort"] for item in frame_data_list])
            if detections_for_sort.size > 0:
                tracks = tracker.update(detections_for_sort)

                for track in tracks:
                    x1, y1, x2, y2, track_id = track
                    video_tracking_results.append({
                        "frame_id": frame_id,
                        "track_id": int(track_id),
                        "bbox": [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
                    })

        gt_by_frame = defaultdict(list)
        for frame_id, frame_data_list in video_annotations_by_frame.items():
            for ann_item in frame_data_list:
                gt_by_frame[frame_id].append((ann_item["cell_id"], ann_item["bbox_gt"]))

        pred_by_frame = defaultdict(list)
        for trk in video_tracking_results: # This was the corrected line.
            pred_by_frame[trk["frame_id"]].append((trk["track_id"], trk["bbox"]))

        all_frames_in_video = sorted(list(set(list(gt_by_frame.keys()) + list(pred_by_frame.keys()))))

        for frame_id in all_frames_in_video:
            gt = gt_by_frame.get(frame_id, [])
            pred = pred_by_frame.get(frame_id, [])

            gt_ids = [g[0] for g in gt]
            gt_boxes = [g[1] for g in gt]

            pred_ids = [p[0] for p in pred]
            pred_boxes = [p[1] for p in pred]

            if gt_boxes or pred_boxes:
                if gt_boxes and pred_boxes:
                    distances = compute_iou_matrix(gt_boxes, pred_boxes)
                    distances[distances > (1 - 0.5)] = np.nan
                    acc.update(gt_ids, pred_ids, distances)
                else:
                    acc.update(gt_ids, pred_ids, np.empty((len(gt_ids), len(pred_ids))))
                total_processed_frames += 1

        del video_annotations_by_frame
        del video_tracking_results
        del gt_by_frame
        del pred_by_frame
        gc.collect()

    print(f"--- Finished processing videos for {name_of_run} ---")

    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=name_of_run)

    print(f"✅ Run {name_of_run}: Processed {total_processed_frames} frames.")

    return summary, name_of_run, total_processed_frames

# === Load Annotations (once) ===
print("Loading initial annotations for mapping (once for all runs)...")
with ANN_FILE.open("r") as f:
    data = json.load(f)

annotations_full = data["annotations"]
images = data["images"]
image_info_by_id = {img["id"]: (img["video_id"], img["frame_id"]) for img in images}
all_video_ids_from_annotations = sorted(list(set(img["video_id"] for img in images)))

del data
gc.collect()

# === Define multiple parameter sets ===
parameter_sets = [
    {
        'name': 'Default_Params_Folders_1_10',
        'MAX_AGE': 1,
        'MIN_HITS': 3,
        'IOU_THRESHOLD': 0.3,
        'START_FOLDER': 1,
        'END_FOLDER': 10
    },
    {
        'name': 'Relaxed_IOU_Folders_1_10',
        'MAX_AGE': 1,
        'MIN_HITS': 3,
        'IOU_THRESHOLD': 0.5, # Increased IOU threshold (more relaxed for association)
        'START_FOLDER': 1,
        'END_FOLDER': 10
    },
    {
        'name': 'HigherMinHits_Folders_1_10',
        'MAX_AGE': 1,
        'MIN_HITS': 5, # Higher min hits for confirmed track
        'IOU_THRESHOLD': 0.3,
        'START_FOLDER': 1,
        'END_FOLDER': 10
    },
    {
        'name': 'Different_Folders_11_20',
        'MAX_AGE': 1,
        'MIN_HITS': 3,
        'IOU_THRESHOLD': 0.3,
        'START_FOLDER': 11, # Different set of folders
        'END_FOLDER': 20
    }
]

# === Run all parameter sets and store results ===
all_results_summaries = {}
all_processed_frames_counts = {}

for params in parameter_sets:
    summary_df, run_name, processed_frames = run_tracking_and_evaluation(
        params, annotations_full, image_info_by_id, all_video_ids_from_annotations
    )
    all_results_summaries[run_name] = summary_df
    all_processed_frames_counts[run_name] = processed_frames

    # Optionally, clear tracker's internal state if not explicitly done by creating new object
    # (SORT already creates a new tracker per video, so this is about global state if any)
    gc.collect() # Force garbage collection between runs to free up memory from the previous run

# === Print all stored summaries ===
print("\n=== All Run Summaries ===")
for run_name, summary_df in all_results_summaries.items():
    print(f"\n--- Summary for: {run_name} ---")
    print(mm.io.render_summary(summary_df, formatters=mm.metrics.create().formatters, namemap=mm.io.motchallenge_metric_names))
    print(f"Total frames processed for this run: {all_processed_frames_counts[run_name]}")

print("\n--- All Done! ---")

In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
from collections import defaultdict
from sort.sort import Sort
import motmetrics as mm
import gc

DATASET_DIR = Path("D:/Maxi/Cell_Tracking_Project_6/dataset_jpg/dataset")
ANN_FILE = DATASET_DIR / "annotations.json"

# === Custom IoU matrix (remains the same) ===
def compute_iou_matrix(gt_boxes, pred_boxes):
    iou_matrix = np.zeros((len(gt_boxes), len(pred_boxes)), dtype=np.float32)
    for i, (x1, y1, w1, h1) in enumerate(gt_boxes):
        xa1, ya1, xa2, ya2 = x1, y1, x1 + w1, y1 + h1
        area_gt = w1 * h1

        for j, (x2, y2, w2, h2) in enumerate(pred_boxes):
            xb1, yb1, xb2, yb2 = x2, y2, x2 + w2, y2 + h2
            area_pred = w2 * h2

            inter_x1 = max(xa1, xb1)
            inter_y1 = max(ya1, yb1)
            inter_x2 = min(xa2, xb2)
            inter_y2 = min(ya2, yb2)

            inter_w = max(0, inter_x2 - inter_x1)
            inter_h = max(0, inter_y2 - inter_y1)
            inter_area = inter_w * inter_h

            union = area_gt + area_pred - inter_area
            iou = inter_area / union if union > 0 else 0.0
            iou_matrix[i, j] = 1.0 - iou
    return iou_matrix

# === Main processing function ===
def run_tracking_and_evaluation(params, annotations_full, image_info_by_id, all_video_ids_from_annotations):
    """
    Runs the SORT tracking and motmetrics evaluation with given parameters.

    Args:
        params (dict): A dictionary containing 'name', 'MAX_AGE', 'MIN_HITS', 'IOU_THRESHOLD',
                       'START_FOLDER', 'END_FOLDER'.
        annotations_full (list): All annotations loaded from the JSON file.
        image_info_by_id (dict): Mapping from image_id to (video_id, frame_id).
        all_video_ids_from_annotations (list): List of all unique video IDs from annotations.

    Returns:
        tuple: (acc, name_of_run, total_processed_frames)
               acc: The MOTAccumulator object containing all events for this run.
               name_of_run: Name of the current parameter set
               total_processed_frames: Number of unique frames processed
    """

    name_of_run = params['name']
    MAX_AGE = params['MAX_AGE']
    MIN_HITS = params['MIN_HITS']
    IOU_THRESHOLD = params['IOU_THRESHOLD']
    START_FOLDER = params['START_FOLDER']
    END_FOLDER = params['END_FOLDER']

    print(f"\n--- Starting run: {name_of_run} ---")
    print(f"Parameters: MAX_AGE={MAX_AGE}, MIN_HITS={MIN_HITS}, IOU_THRESHOLD={IOU_THRESHOLD}")
    print(f"Folders: {START_FOLDER} to {END_FOLDER}")

    acc = mm.MOTAccumulator(auto_id=True)

    video_ids_to_process = []
    for vid_id in all_video_ids_from_annotations:
        try:
            vid_num = int(vid_id)
            if START_FOLDER <= vid_num <= END_FOLDER:
                 video_ids_to_process.append(vid_id)
        except ValueError:
            continue

    # print(f"Total videos to process for this run: {len(video_ids_to_process)}")

    total_frames_updated = 0 # Initialize a counter for frames updated

    for video_idx, video_id in enumerate(video_ids_to_process):
        print(f"Processing video {video_idx + 1}/{len(video_ids_to_process)}: Video ID {video_id}", end='\r')

        video_annotations_by_frame = defaultdict(list)
        video_tracking_results = []

        for ann in annotations_full:
            img_id = ann["image_id"]
            if img_id in image_info_by_id:
                vid_id_from_img, frame_id = image_info_by_id[img_id]
                if vid_id_from_img == video_id:
                    x, y, w, h = ann["bbox"]
                    x1, y1 = x - w / 2, y - h / 2
                    x2, y2 = x1 + w, y1 + h
                    bbox_for_sort = [x1, y1, x2, y2, 1.0]
                    video_annotations_by_frame[frame_id].append({
                        "cell_id": ann["cell_id"],
                        "bbox_gt": [x1, y1, w, h],
                        "bbox_sort": bbox_for_sort
                    })

        tracker = Sort(max_age=MAX_AGE, min_hits=MIN_HITS, iou_threshold=IOU_THRESHOLD)
        sorted_frames = sorted(video_annotations_by_frame.items())

        for frame_id, frame_data_list in sorted_frames:
            detections_for_sort = np.array([item["bbox_sort"] for item in frame_data_list])
            if detections_for_sort.size > 0:
                tracks = tracker.update(detections_for_sort)

                for track in tracks:
                    x1, y1, x2, y2, track_id = track
                    video_tracking_results.append({
                        "frame_id": frame_id,
                        "track_id": int(track_id),
                        "bbox": [float(x1), float(y1), float(x2 - x1), float(y2 - y1)]
                    })

        gt_by_frame = defaultdict(list)
        for frame_id, frame_data_list in video_annotations_by_frame.items():
            for ann_item in frame_data_list:
                gt_by_frame[frame_id].append((ann_item["cell_id"], ann_item["bbox_gt"]))

        pred_by_frame = defaultdict(list)
        for trk in video_tracking_results:
            pred_by_frame[trk["frame_id"]].append((trk["track_id"], trk["bbox"]))

        all_frames_in_video = sorted(list(set(list(gt_by_frame.keys()) + list(pred_by_frame.keys()))))

        for frame_id in all_frames_in_video:
            gt = gt_by_frame.get(frame_id, [])
            pred = pred_by_frame.get(frame_id, [])

            gt_ids = [g[0] for g in gt]
            gt_boxes = [g[1] for g in gt]

            pred_ids = [p[0] for p in pred]
            pred_boxes = [p[1] for p in pred]

            if gt_boxes or pred_boxes:
                if gt_boxes and pred_boxes:
                    distances = compute_iou_matrix(gt_boxes, pred_boxes)
                    distances[distances > (1 - 0.5)] = np.nan
                    acc.update(gt_ids, pred_ids, distances)
                else:
                    acc.update(gt_ids, pred_ids, np.empty((len(gt_ids), len(pred_ids))))
                total_frames_updated += 1

    # print(f"--- Finished processing videos for {name_of_run} ---")
    print(f"✅ Run {name_of_run}: Processed {total_frames_updated} frames.")

    return acc, name_of_run, total_frames_updated # Return the acc object directly

# === Load Annotations (once) ===
print("Loading initial annotations for mapping (once for all runs)...")
with ANN_FILE.open("r") as f:
    data = json.load(f)

annotations_full = data["annotations"]
images = data["images"]
image_info_by_id = {img["id"]: (img["video_id"], img["frame_id"]) for img in images}
all_video_ids_from_annotations = sorted(list(set(img["video_id"] for img in images)))

del data
gc.collect()
print("✅ Finished loading")

Loading initial annotations for mapping (once for all runs)...


0

In [3]:
# === Define multiple parameter sets ===
parameter_sets = [
    {
        'name': 'Default',
        'MAX_AGE': 1,
        'MIN_HITS': 3,
        'IOU_THRESHOLD': 0.3,
        'START_FOLDER': 1,
        'END_FOLDER': 50
    },
    {
        'name': 'Relaxed_IOU',
        'MAX_AGE': 1,
        'MIN_HITS': 3,
        'IOU_THRESHOLD': 0.5,
        'START_FOLDER': 1,
        'END_FOLDER': 50
    },
    {
        'name': 'HigherMinHits',
        'MAX_AGE': 1,
        'MIN_HITS': 5,
        'IOU_THRESHOLD': 0.3,
        'START_FOLDER': 1,
        'END_FOLDER': 50
    },
    # {
    #     'name': 'Different_Folders',
    #     'MAX_AGE': 1,
    #     'MIN_HITS': 3,
    #     'IOU_THRESHOLD': 0.3,
    #     'START_FOLDER': 11,
    #     'END_FOLDER': 20
    # }
]

# === Run all parameter sets and store results ===
all_accumulators = {} # Storing MOTAccumulator
all_processed_frames_counts = {}

for params in parameter_sets:
    acc_obj, run_name, processed_frames = run_tracking_and_evaluation(
        params, annotations_full, image_info_by_id, all_video_ids_from_annotations
    )
    all_accumulators[run_name] = acc_obj
    all_processed_frames_counts[run_name] = processed_frames

    # Garbage Collection
    gc.collect()

# === Print all stored summaries in one go ===
print("\n=== Combined All Run Summaries ===")
mh = mm.metrics.create()

summary = mh.compute_many(
    list(all_accumulators.values()),
    metrics=mm.metrics.motchallenge_metrics,
    names=list(all_accumulators.keys()),
    generate_overall=False
)

# Add additional info the summary rendering
params_df = pd.DataFrame(parameter_sets).set_index('name')
params_to_display = ['MAX_AGE', 'MIN_HITS', 'IOU_THRESHOLD', 'START_FOLDER', 'END_FOLDER']
params_df_selected = params_df[params_to_display]
summary.index.name = 'name'
combined_summary = summary.join(params_df_selected)

print(mm.io.render_summary(
    combined_summary,
    formatters=mh.formatters,
    namemap=mm.io.motchallenge_metric_names
))

print("\n--- Total Frames Processed Per Run ---")
total_count = 0
for run_name, count in all_processed_frames_counts.items():
    print(f"{run_name}: {count} frames")
    total_count += count
print(f"➡️ TOTAL: {total_count} frames")


--- Starting run: Default ---
Parameters: MAX_AGE=1, MIN_HITS=3, IOU_THRESHOLD=0.3
Folders: 1 to 50
✅ Run Default: Processed 7500 frames.

--- Starting run: Relaxed_IOU ---
Parameters: MAX_AGE=1, MIN_HITS=3, IOU_THRESHOLD=0.5
Folders: 1 to 50
✅ Run Relaxed_IOU: Processed 7500 frames.

--- Starting run: HigherMinHits ---
Parameters: MAX_AGE=1, MIN_HITS=5, IOU_THRESHOLD=0.3
Folders: 1 to 50
✅ Run HigherMinHits: Processed 7500 frames.

=== Combined All Run Summaries ===
              IDF1  IDP  IDR  Rcll   Prcn  GT  MT  PT ML  FP    FN  IDs    FM  MOTA  MOTP IDt  IDa IDm  MAX_AGE  MIN_HITS  IOU_THRESHOLD  START_FOLDER  END_FOLDER
name                                                                                                                                                              
Default       9.1% 9.6% 8.7% 90.6%  99.7% 245 203  38  4 534 21322 5861  5621 87.8% 0.107 570 5301  42        1         3            0.3             1          50
Relaxed_IOU   6.7% 7.5% 6.1% 80.9% 100